# Reinforcement Learning

In [4]:
#!pip install gym
import gym  
import random  

random.seed(1234) 

streets = gym.make("Taxi-v3", render_mode = "ansi").env  # cria o ambiente Taxi-v3, que simula um problema de táxi e ansi define o modo de renderização textual
streets.reset()  # reseta o ambiente
print("\n"+streets.render())  # exibe em formato textual


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [5]:
initial_state = streets.encode(2, 3, 2, 0)  # codifica um estado inicial específico, onde:
# 2 é a linha do táxi, 3 é a coluna do táxi, 2 é a localização do passageiro, e 0 é o destino.

streets.s = initial_state  # iguala o estado inicial ao estado codificado.

streets.render()  # ambiente para visualizar o estado inicial configurado.

'+---------+\n|\x1bR\x1b: | : :G|\n| : | : : |\n| : : : : |\n| | : | : |\n|Y| : |\x1b\x1bB\x1b\x1b: |\n+---------+\n\n'

In [6]:
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

In [11]:
for taxi_run in range(epochs):
    state = streets.reset()
    done = False

    while not done:
        if isinstance(state, tuple):
            state_index, info = state
            action_mask = info.get('action_mask', np.ones(streets.action_space.n, dtype=int))
        else:
            state_index = state
            action_mask = np.ones(streets.action_space.n, dtype=int)

        random_value = random.uniform(0, 1)
        if random_value < exploration:
            valid_actions = np.where(action_mask == 1)[0]
            action = np.random.choice(valid_actions)
        else:
            q_values = q_table[state_index]
            valid_q_values = q_values * action_mask
            action = np.argmax(valid_q_values)

        result = streets.step(action)
        if len(result) == 4:
            next_state, reward, done, info = result
        elif len(result) == 5:
            next_state, reward, done, truncated, info = result
            done = done or truncated  
        else:
            raise ValueError(f"streets.step(action) retornou um número inesperado de valores: {len(result)}")

        if isinstance(next_state, tuple):
            next_state = next_state[0]

        prev_q = q_table[state_index, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        q_table[state_index, action] = new_q

        state = next_state


In [12]:
q_table[initial_state] 

array([ -2.42159969,  -2.41412014,  -2.40174171,  -2.3639511 ,
       -11.37731913,  -6.36445316])

In [16]:
#faz a simulacao das viagens

from IPython.display import clear_output
from time import sleep

for tripnum in range(1, 11):
    state = streets.reset()
   
    done = False
    trip_length = 0
    
    while not done and trip_length < 25:
        action = np.argmax(q_table[state])
        next_state, reward, done, info = streets.step(action)
        clear_output(wait=True)
        print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        print(streets.render(mode='ansi'))
        sleep(.5)
        state = next_state
        trip_length += 1
        
    sleep(2)
    

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices